<a href="https://colab.research.google.com/github/sweety1920/FMML-LABS-4/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging the validation accuracy across multiple splits generally **gives more consistent and reliable results**. This approach is part of a technique known as **cross-validation**, and it helps mitigate some of the variability or bias that might occur when evaluating a model on a single train-test split.

Here’s a deeper explanation of why averaging across multiple splits provides more consistent results:

### 1. **Reduction of Variance in Model Evaluation**
   - **Single Train-Test Split**: When you evaluate a model on a single train-test split, the performance might be overly optimistic or pessimistic depending on the specific characteristics of the data that happened to be in the training and validation sets. For example, if the validation set happens to be harder (or easier) than average, you might get an unrepresentative estimate of the model's true performance.
   - **Multiple Splits (Cross-Validation)**: By performing cross-validation, you use multiple different splits of the data for training and testing. For instance, in **k-fold cross-validation**, the data is split into **k** subsets, and the model is trained **k times**, each time using a different subset as the validation set and the remaining subsets for training. The performance scores (e.g., accuracy) from all **k** splits are then averaged to provide a more robust and reliable estimate of the model’s performance.

### 2. **Better Generalization**
   - By averaging over multiple splits, the model is evaluated across different subsets of the data. This helps ensure that the model is **not overfitting** to a particular subset or becoming too sensitive to specific data points. It forces the model to generalize better across various unseen portions of the data.
   - This is especially important when the data is small or if the data has a high degree of variability. Cross-validation reduces the likelihood of a model being evaluated on an unrepresentative test set, which could lead to misleading results.

### 3. **Mitigation of Overfitting**
   - **Overfitting** happens when a model learns the noise or specific patterns from the training data that don't generalize well to unseen data. Using multiple splits helps in **averaging out the effects of overfitting** to any specific subset of the data, as the model is evaluated on different data points each time.
   - For example, if the model overfits the first split but performs well on others, averaging the results will give a more honest performance estimate, helping to detect overfitting and ensuring the model is more likely to generalize to new data.

### 4. **Improved Model Tuning**
   - Cross-validation allows for more reliable comparison of different models or hyperparameter settings. For instance, when tuning hyperparameters (like learning rate, batch size, or number of layers in a neural network), the validation accuracy from a single split might mislead you. However, if you use cross-validation, the model's performance is averaged across multiple different datasets, making it easier to detect whether a particular configuration consistently performs well or poorly.

### 5. **Handling Data Imbalance**
   - When the data is imbalanced (e.g., there are many more examples of one class than another), a single train-test split might lead to an unrepresentative validation set, which could affect the accuracy. However, by averaging the performance over multiple splits (and using stratified k-fold cross-validation, if necessary), you ensure that each class is well-represented in both the training and validation sets. This gives a more balanced and consistent measure of performance.

### 6. **Potential Limitations**
While averaging across multiple splits gives more consistent results, it’s important to note that:
   - **Computational Cost**: Cross-validation can be more computationally expensive, especially with a large dataset or if you're using a complex model (like deep learning). This is because you are training the model multiple times, which requires more time and resources.
   - **Training Data Leakage**: If there’s data leakage between the training and validation sets in any split, the results might still be biased. Careful cross-validation procedures should be followed to avoid this.

### Common Cross-Validation Approaches:
   1. **K-Fold Cross-Validation**: The data is split into **k** subsets (folds). The model is trained **k** times, with each fold used as the validation set once, and the remaining folds as the training set. The performance metrics are averaged over the **k** iterations. Typical values for **k** are 5 or 10.
   2. **Stratified K-Fold Cross-Validation**: In stratified k-fold cross-validation, the data is split such that the proportion of each class in each fold is approximately the same as in the entire dataset. This is especially important in classification tasks with imbalanced classes.
   3. **Leave-One-Out Cross-Validation (LOOCV)**: A special case where **k** equals the number of data points. In each iteration, one data point is used as the validation set, and the rest of the data points are used for training. This can be very computationally expensive for large datasets but is sometimes used when the dataset is small.

### Example:
Let’s assume you have a dataset with 1000 samples, and you perform 5-fold cross-validation. The model will be trained 5 times, each time on 800 samples and validated on 200 samples (the fold used for validation). The accuracy from each fold is calculated, and the final performance is the **average** of these 5 accuracy scores.

This will give a more robust estimate of the model’s performance than evaluating it on a single split, where the choice of training and validation sets can significantly influence the results.

---

### Conclusion:
Averaging the validation accuracy across multiple splits helps to **smooth out fluctuations** in the performance and gives a **more reliable, consistent estimate** of how well a model will perform on unseen data. It reduces bias and variance, helps with model tuning, and provides a better generalization to real-world use cases. For this reason, cross-validation is a widely-used method in machine learning and statistical modeling to evaluate and compare model performance.

2. Does it give more accurate estimate of test accuracy?
To evaluate whether a method gives a more accurate estimate of test accuracy, you need to consider several factors:

1. **Methodology of Evaluation**:
   - **Holdout Validation**: This involves splitting the dataset into training and testing subsets. The estimate can be inaccurate if the test set is not representative of the dataset or if the sample size is small.
   - **Cross-Validation**: Techniques like k-fold cross-validation typically provide more robust accuracy estimates by using different parts of the dataset for training and testing multiple times. This helps reduce variance due to data splitting.
   - **Bootstrap Resampling**: This method involves repeatedly sampling with replacement to estimate the accuracy. It can provide insight into variability but may not be as straightforward as cross-validation.

2. **Size and Representativeness of the Dataset**:
   - Smaller datasets can lead to higher variability in test accuracy estimates. Larger, well-represented datasets generally yield more reliable estimates.

3. **Bias-Variance Tradeoff**:
   - Simpler methods may have higher bias but lower variance. Cross-validation balances this tradeoff better than simple holdout validation.

4. **Overfitting Concerns**:
   - When selecting models or hyperparameters, it’s essential to use a separate validation set or nested cross-validation. Evaluating on the same dataset used for model selection leads to overestimated accuracy.

5. **Real-World Distribution**:
   - The estimate's accuracy depends on how well the test set mimics the real-world data distribution. If there’s a mismatch, even advanced methods might not give an accurate reflection of real-world performance.

If your question refers to a specific method or context, feel free to provide more details for a tailored response.


3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
The number of iterations in an evaluation process (such as cross-validation or bootstrap resampling) significantly affects the reliability of the estimate. Here’s a detailed look at the effects:

### **Effects of Increasing the Number of Iterations**
1. **Reduced Variance**:
   - As the number of iterations increases, the variance of the accuracy estimate decreases. Each additional iteration provides more data points, resulting in a more stable and reliable estimate.

2. **Better Representation of Dataset Variability**:
   - Higher iterations ensure that more combinations of the data are evaluated, particularly in methods like k-fold cross-validation with repetition or bootstrapping. This helps capture variations in the data and provides a better overall estimate of model performance.

3. **Improved Robustness**:
   - The estimate becomes less sensitive to any particular random split of the data. This is especially important for datasets with uneven distributions or outliers.

4. **Diminishing Returns**:
   - While increasing iterations improves the estimate, the improvement diminishes after a certain point. Beyond a threshold, the gain in accuracy precision might not justify the additional computational cost.

---

### **When Higher Iterations Help**
- **Small Datasets**: For small datasets, a higher number of iterations compensates for the lack of data by making better use of the available samples.
- **High Variability**: If the performance varies significantly across different splits of the data, more iterations provide a clearer picture of the model’s average performance.

---

### **When Higher Iterations Might Not Help**
1. **Overfitting on Validation Procedure**:
   - If the procedure becomes excessively detailed (e.g., repeating cross-validation too many times), it might reflect noise in the data rather than true performance.

2. **Computational Cost**:
   - Higher iterations can be computationally expensive, particularly for large datasets or complex models.

3. **Bias in Evaluation Design**:
   - If there’s inherent bias (e.g., data leakage or unrepresentative splits), increasing iterations won't fix the underlying issue.

---

### **Key Takeaways**
- Higher iterations generally lead to better estimates due to reduced variance and improved stability.
- The benefit of additional iterations diminishes over time, with a practical balance between computational cost and accuracy improvement being ideal.
- For most scenarios, repeated k-fold cross-validation (e.g., 10-fold repeated 3–5 times) provides a good balance between accuracy and efficiency.

4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Using a very small training or validation dataset poses challenges, even if you increase the number of iterations. Here's an analysis based on the previous discussion:

---

### **Can Increasing Iterations Compensate for a Small Dataset?**

1. **Advantages of Higher Iterations**:
   - **Maximized Use of Available Data**:
     - Techniques like repeated k-fold cross-validation or bootstrapping use the data multiple times, leveraging limited data more effectively.
   - **Reduced Variance**:
     - By increasing the number of iterations, you can smooth out fluctuations caused by small dataset variability, providing a more stable estimate of model performance.

2. **Limitations of Small Datasets**:
   - **Lack of Representativeness**:
     - If the training dataset is too small, it may not adequately represent the underlying data distribution, leading to biased model performance, no matter how many iterations are used.
   - **Overfitting Risk**:
     - Small training datasets make models prone to overfitting, as they may memorize the training examples rather than learning generalizable patterns.
   - **Validation Bias**:
     - With a small validation set, performance metrics can be overly optimistic or pessimistic, as they depend heavily on the specific data points included.

---

### **Strategies for Small Datasets**:
1. **Data Augmentation**:
   - If applicable, use techniques to synthetically increase the dataset size, such as rotations, scaling, or noise addition (common in image and text domains).
   
2. **Simpler Models**:
   - Use models with fewer parameters that are less prone to overfitting on small datasets (e.g., linear models or regularized regression).

3. **Leave-One-Out Cross-Validation (LOOCV)**:
   - This is particularly useful for small datasets as it maximizes the training data while still providing an independent validation estimate for each point. However, it can be computationally expensive.

4. **Transfer Learning**:
   - If the dataset is small, leveraging pre-trained models (for example, in deep learning) can mitigate the lack of sufficient training data.

5. **Bayesian or Probabilistic Approaches**:
   - Bayesian methods can better handle uncertainty in predictions caused by small datasets.

---

### **Key Takeaways**:
- **Increasing iterations** helps mitigate variability and improve the reliability of the performance estimate for small datasets.
- However, it **cannot compensate** for the inherent limitations of a small dataset, such as lack of representativeness or overfitting risks.
- To handle small datasets effectively, combine higher iterations with data augmentation, simpler models, or external information (e.g., pre-trained models or priors).

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.